<a href="https://colab.research.google.com/github/Nibi2003/Transfomer_keypoint/blob/main/Codigos/Usando_extratores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
!pip install mediapipe


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 23.3 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.15.0 requires protobuf<4.21,>=3.20.3; python_version < "3.11", but you have protobuf 4.25.3 which is incompatible.


# O código abaixo utiliza o extrator do midia pipe, requer um diretorio de origem e um de destino. É necessário colocar os kp em uma lista, já que não é um array homogeneo considerando kps vazios

In [15]:
import cv2
import numpy as np
import os
import mediapipe as mp

class MediaPipeKeypoints:
    def __init__(self):
        self.mp_pose = mp.solutions.pose
        self.pose = self.mp_pose.Pose()
        self.mp_drawing = mp.solutions.drawing_utils

    def detect(self, frame):
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = self.pose.process(image_rgb)
        return results

    def draw(self, frame, results):
        annotated_image = frame.copy()
        self.mp_drawing.draw_landmarks(annotated_image, results.pose_landmarks, self.mp_pose.POSE_CONNECTIONS)
        return annotated_image

# Diretórios
original_videos_dir = '/content/drive/MyDrive/GMA/Vídeos bebês segmentos 2023/típico_cortado_padrao_1920x1080/split_videos/'
output_base_dir = '/content/drive/MyDrive/GMA/Vídeos bebês segmentos 2023/keypoints_extraidos/MidiaPipe/0/'

# Criar diretório de saída, se não existir
if not os.path.exists(output_base_dir):
    os.makedirs(output_base_dir)

# Inicializar o detector de keypoints
mp_keypoints = MediaPipeKeypoints()

# Função para processar vídeos e salvar keypoints
def process_videos():
    for video_file in os.listdir(original_videos_dir):
        video_path = os.path.join(original_videos_dir, video_file)
        if os.path.isfile(video_path):
            print(f'Processando {video_file}...')
            # Abrir vídeo
            cap = cv2.VideoCapture(video_path)
            keypoints_list = []

            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break

                # Detectar keypoints
                results = mp_keypoints.detect(frame)
                if results.pose_landmarks:
                    keypoints = [(lm.x, lm.y) for lm in results.pose_landmarks.landmark]
                else:
                    keypoints = [None] * 33  # Assumindo 33 pontos no modelo Pose

                keypoints_list.append(keypoints)

            cap.release()

            # Ajustar formato dos dados
            keypoints_array = np.array(keypoints_list, dtype=object)  # Usar dtype=object para lidar com None
            keypoints_array = np.array([[kp if kp is not None else (None, None) for kp in frame] for frame in keypoints_array])

            # Salvar keypoints em .npy
            output_file = os.path.join(output_base_dir, os.path.splitext(video_file)[0] + '_keypoints.npy')
            np.save(output_file, keypoints_array, allow_pickle=True)
            print(f'Salvo keypoints em {output_file}')

process_videos()



Processando VID-20231016-WA0060_1_part_1.mp4...
Salvo keypoints em /content/drive/MyDrive/GMA/Vídeos bebês segmentos 2023/keypoints_extraidos/MidiaPipe/0/VID-20231016-WA0060_1_part_1_keypoints.npy
Processando VID-20231016-WA0054_15_part_1.mp4...
Salvo keypoints em /content/drive/MyDrive/GMA/Vídeos bebês segmentos 2023/keypoints_extraidos/MidiaPipe/0/VID-20231016-WA0054_15_part_1_keypoints.npy
Processando VID-20231016-WA0054_16_part_1.mp4...
Salvo keypoints em /content/drive/MyDrive/GMA/Vídeos bebês segmentos 2023/keypoints_extraidos/MidiaPipe/0/VID-20231016-WA0054_16_part_1_keypoints.npy
Processando VID-20231016-WA0068_1_part_1.mp4...
Salvo keypoints em /content/drive/MyDrive/GMA/Vídeos bebês segmentos 2023/keypoints_extraidos/MidiaPipe/0/VID-20231016-WA0068_1_part_1_keypoints.npy
Processando VID-20231016-WA0068_3_part_1.mp4...
Salvo keypoints em /content/drive/MyDrive/GMA/Vídeos bebês segmentos 2023/keypoints_extraidos/MidiaPipe/0/VID-20231016-WA0068_3_part_1_keypoints.npy
Processando

> Verifica se o código deu certo

In [16]:

npy_dir = '/content/drive/MyDrive/GMA/Vídeos bebês segmentos 2023/keypoints_extraidos/MidiaPipe/0/'

# Percorrer o diretório e carregar cada arquivo .npy
for npy_file in os.listdir(npy_dir):
    if npy_file.endswith('.npy'):
        file_path = os.path.join(npy_dir, npy_file)
        data = np.load(file_path, allow_pickle=True)
        print(f'Arquivo: {npy_file}, Formato: {data.shape}')


Arquivo: VID-20231016-WA0060_1_part_1_keypoints.npy, Formato: (900, 33, 2)
Arquivo: VID-20231016-WA0054_15_part_1_keypoints.npy, Formato: (900, 33, 2)
Arquivo: VID-20231016-WA0054_16_part_1_keypoints.npy, Formato: (900, 33, 2)
Arquivo: VID-20231016-WA0068_1_part_1_keypoints.npy, Formato: (900, 33, 2)
Arquivo: VID-20231016-WA0068_3_part_1_keypoints.npy, Formato: (900, 33, 2)
Arquivo: VID-20231016-WA0080_4_part_1_keypoints.npy, Formato: (900, 33, 2)
Arquivo: VID-20231016-WA0080_4_part_2_keypoints.npy, Formato: (900, 33, 2)
Arquivo: VID-20231016-WA0080_6_part_1_keypoints.npy, Formato: (900, 33, 2)
Arquivo: VID-20231016-WA0080_6_part_2_keypoints.npy, Formato: (900, 33, 2)
Arquivo: VID-20231016-WA0080_6_part_3_keypoints.npy, Formato: (900, 33, 2)
Arquivo: VID-20231016-WA0078_1_part_1_keypoints.npy, Formato: (900, 33, 2)
Arquivo: VID-20231016-WA0078_1_part_2_keypoints.npy, Formato: (900, 33, 2)
Arquivo: VID-20231016-WA0078_1_part_3_keypoints.npy, Formato: (900, 33, 2)
Arquivo: VID-20231016-W